# ERA5 Data Assimilation Example

In [ ]:
from pathlib import Path
import sys

repo_root = Path.cwd().resolve()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent

if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

print(f'Repo root: {repo_root}')


## 1) Experiment Configuration


In [ ]:
SAVE_PREFIX = 'fullobs_direct_era5_'
WINDOW_LENGTH = 4
OBSERVATION_SCHEDULE = list(range(WINDOW_LENGTH))

DA_CONFIG = {
    'obs_ratio': 0.15,
    'obs_noise_std': 0.00,
    'observation_schedule': OBSERVATION_SCHEDULE,
    'observation_variance': None,
    'window_length': WINDOW_LENGTH,
    'num_runs': 5,
    'start_T': 0,
    'save_prefix': SAVE_PREFIX,
}

DA_CONFIG

## 2) Unified Data Assimilation for all baselines


In [ ]:
from src.assimilation.era5_full_observation import run_all_models

summary = run_all_models(**DA_CONFIG)
summary


## 3) channel-wise DA metic（MSE / RRMSE / SSIM）


In [ ]:
from IPython.display import Image, display

from src.plot.era5_DA import (
    load_model_metrics,
    print_time_summary,
    build_comparison_figure,
)

model_data = load_model_metrics(repo_root, prefix=SAVE_PREFIX)
model_names = list(model_data.keys())
print_time_summary(repo_root, model_names, prefix=SAVE_PREFIX)

obs_schedule_to_draw = OBSERVATION_SCHEDULE if 'interobs' in SAVE_PREFIX.lower() else None

figures_dir = repo_root / 'results' / 'Comparison' / 'figures'
output_da = figures_dir / f'{SAVE_PREFIX if SAVE_PREFIX else "default_"}era5_DA_comparison.png'

fig = build_comparison_figure(
    model_data=model_data,
    output_path=output_da,
    observation_schedule=obs_schedule_to_draw,
)

display(Image(filename=str(output_da)))
fig


## 4) Animation Example


In [ ]:
from IPython.display import Image, display

from src.plot.era5_gif_separate_background import make_era5_da_gif_separate

DATA_PATH = str(repo_root / 'data/ERA5/ERA5_data/test_seq_state.h5')
MIN_PATH = str(repo_root / 'data/ERA5/ERA5_data/min_val.npy')
MAX_PATH = str(repo_root / 'data/ERA5/ERA5_data/max_val.npy')
RESULTS_ROOT = str(repo_root / 'results')
FIGURE_DIR = str(repo_root / 'results/Comparison/figures')

result_filename = f'{SAVE_PREFIX}multi.npy'
rollout_filename = f'{SAVE_PREFIX}multi_original.npy'

gif_paths = make_era5_da_gif_separate(
    data_path=DATA_PATH,
    min_path=MIN_PATH,
    max_path=MAX_PATH,
    results_root=RESULTS_ROOT,
    result_filename=result_filename,
    rollout_filename=rollout_filename,
    out_dir=FIGURE_DIR,
    start_t=DA_CONFIG['start_T'],
    start_datetime_str='2018-01-01 06:00',
    hours_per_frame=6,
)

CHANNEL_TO_SHOW = 0  # 0~4
display(Image(filename=gif_paths[CHANNEL_TO_SHOW]))
gif_paths


## 5) Power Spectrum Distribution


In [ ]:
from IPython.display import Image, display

from src.plot.era5_PSD import make_spatial_power_spectrum_grid

psd_out = repo_root / 'results/Comparison/figures' / f'{SAVE_PREFIX}era5_spectrum_grid.png'

psd_path = make_spatial_power_spectrum_grid(
    data_path=DATA_PATH,
    min_path=MIN_PATH,
    max_path=MAX_PATH,
    results_root=RESULTS_ROOT,
    da_filename=f'{SAVE_PREFIX}multi.npy',
    noda_filename=f'{SAVE_PREFIX}multi_original.npy',
    da_subdir='DA',
    noda_subdir='DA',
    out_path=str(psd_out),
    start_t=DA_CONFIG['start_T'],
    row_label_fontsize=20,
    col_label_fontsize=20,
    tick_fontsize=9,
    sup_fontsize=13,
    legend_fontsize=14,
    gt_lw=2.6,
    da_lw=2.4,
    noda_lw=2.4,
)

display(Image(filename=str(psd_path)))
psd_path